# Z3 SAT for N-Queens

In [52]:
from itertools import combinations
from z3 import *
from utils import *
import math
import time
import json

## Encodings: at least one, at most one, exactly one

### Naive pairwise

In [53]:
def at_least_one_np(bool_vars):
    return Or(bool_vars)

def at_most_one_np(bool_vars, name = ""):
    return And([Not(And(pair[0], pair[1])) for pair in combinations(bool_vars, 2)])

def exactly_one_np(bool_vars, name = ""):
    return And(at_most_one_np(bool_vars, ''), at_least_one_np(bool_vars))

### Sequential

In [54]:
def at_least_one_seq(bool_vars):
    return at_least_one_np(bool_vars)

def at_most_one_seq(bool_vars, name=''):
    constraints = []
    n = len(bool_vars)
    s = [Bool(f's_{name}_{i}') for i in range(n-1)]
    constraints.append(Or(Not(bool_vars[0]), s[0]))
    constraints.append(Or(Not(bool_vars[n-1]), Not(s[n-2])))
    for i in range(1, n-1):
        constraints.append(Or(Not(bool_vars[i]), s[i]))
        constraints.append(Or(Not(bool_vars[i]), Not(s[i-1])))
        constraints.append(Or(Not(s[i-1]), s[i]))
    return And(constraints)

def exactly_one_seq(bool_vars, name=''):
    return And(at_most_one_seq(bool_vars, name), at_least_one_seq(bool_vars))

### Bitwise

In [55]:
def toBinary(num, length = None):
    num_bin = bin(num).split('b')[-1]
    if length:
        return '0'*(length - len(num_bin)) + num_bin
    return num_bin

def at_least_one_bw(bool_vars):
    return at_least_one_np(bool_vars)

def at_most_one_bw(bool_vars, name):
    constraints = []
    n = len(bool_vars)
    m = math.ceil(math.log2(n))
    r = [Bool(f'r_{name}_{i}') for i in range(m)]
    binary_representation = [toBinary(i, m) for i in range(n)]
    for i in range(n):
        for j in range(m):
            phi = Not(r[j])
            if binary_representation[i][j] == '1':
                phi = r[j]
            constraints.append(Or(Not(bool_vars[i]), phi))
    return And(constraints)

def exactly_one_bw(bool_vars, name):
    return And(at_most_one_bw(bool_vars, name), at_least_one_bw(bool_vars))

### Heule (it's broken, must be fixed)

In [56]:
def at_least_one_he(bool_vars):
    return at_least_one_np(bool_vars)

def at_most_one_he(bool_vars, name):
    if len(bool_vars) <= 4:
        return at_most_one_np(bool_vars, name)
    else:
        y = Bool(f'y_{name}')
        return And(And(at_most_one_np(bool_vars[:3], name), y), at_most_one_he(bool_vars[3:] + [Not(y)], name+'_'))

def exactly_one_he(bool_vars, name):
    return And(at_most_one_he(bool_vars, name), at_least_one_he(bool_vars))

## Encodings: at most k, at least k, exactly k 

### Naive pairwise

In [57]:
def at_least_k_np(bool_vars, k, name = ""):
    return at_most_k_np([Not(var) for var in bool_vars], len(bool_vars) - k)

def at_most_k_np(bool_vars, k, name = ""):
    return And([Or([Not(x) for x in X]) for X in combinations(bool_vars, k+1)])

def exactly_k_np(bool_vars, k, name = ""):
    return at_most_k_np(bool_vars, k) and at_least_k_np(bool_vars, k)

### Sequential

In [58]:
def at_least_k_seq(bool_vars, k, name):
    return at_least_k_np(bool_vars, k, name)

def at_most_k_seq(bool_vars, k, name):
    constraints = []
    n = len(bool_vars)
    s = [[Bool(f's_{name}_{i}_{j}') for j in range(k)] for i in range(n-1)]
    constraints.append(Or(Not(bool_vars[0]), s[0][0]))
    constraints += [Not(s[0][j]) for j in range(1, k)]
    for i in range(1, n-1):
        constraints.append(Or(Not(bool_vars[i]), s[i][0]))
        constraints.append(Or(Not(s[i-1][0]), s[i][0]))
        constraints.append(Or(Not(bool_vars[i]), s[i][1]))
        for j in range(1,k):
            constraints.append(Or(Not(bool_vars[i]), Not(s[i-1][j-1]), s[i][j]))
            constraints.append(Or(Not(s[i-1][j]), s[i][j]))
        constraints.append(Or(Not(bool_vars[n-1]), Not(s[n-2][k-1])))
    return And(constraints)


def exactly_k_seq(bool_vars, k, name):
    return And(at_most_k_seq(bool_vars, k, name), at_least_k_seq(bool_vars, k, name))

## N-Queens problem

The [n-queens problem](https://en.wikipedia.org/wiki/Eight_queens_puzzle) is the problem of placing n chess queens in a $nxn$ chessboard so that no two queens threaten each other. In light of this, the solution requires that no two queens share the same row, column, or diagonal.

In [59]:
def nqueens_sat(n):
    p = [[Bool(f"x_{i}_{j}") for j in range(n)] for i in range(n)]

    # Create the solver instance
    s = Solver()

    # Exactly one on each row and column
    for i in range(n):
        s.add(exactly_one(p[i], f'row_{i}'))
        s.add(exactly_one([p[j][i] for j in range(n)], f'col_{i}'))
              

    # Add the diagonal constraints
    for i in range(n-1):
        diag_ru_ll = []
        diag_lu_rl = []
        diag_ll_ru = []
        diag_rl_lu = []
        for j in range(n-i):
            diag_ru_ll += [p[i+j][j]]
            diag_lu_rl += [p[n-1-(i+j)][j]]
            diag_ll_ru += [p[i+j][n-j-1]]
            diag_rl_lu += [p[n-1-(i+j)][n-j-1]]
        s.add(at_most_one(diag_ru_ll, f'd1_{i}'))
        s.add(at_most_one(diag_lu_rl, f'd2_{i}'))
        s.add(at_most_one(diag_ll_ru, f'd3_{i}'))
        s.add(at_most_one(diag_rl_lu, f'd4_{i}'))

    if s.check() == sat:
        m = s.model()
        return [(i,j) for i in range(n) for j in range(n) if m.evaluate(p[i][j])]
    else:
        return None
   


## Solving NQueens Examples

In [60]:
%%time
at_most_one = at_most_one_np
at_least_one = at_least_one_np
exactly_one = exactly_one_np
solution = nqueens_sat(14)
print("sol : ", solution)
display_nqueens(solution)

sol :  [(0, 3), (1, 7), (2, 9), (3, 11), (4, 6), (5, 1), (6, 10), (7, 12), (8, 5), (9, 0), (10, 2), (11, 4), (12, 13), (13, 8)]
. . . ♛ . . . . . . . . . . 
. . . . . . . ♛ . . . . . . 
. . . . . . . . . ♛ . . . . 
. . . . . . . . . . . ♛ . . 
. . . . . . ♛ . . . . . . . 
. ♛ . . . . . . . . . . . . 
. . . . . . . . . . ♛ . . . 
. . . . . . . . . . . . ♛ . 
. . . . . ♛ . . . . . . . . 
♛ . . . . . . . . . . . . . 
. . ♛ . . . . . . . . . . . 
. . . . ♛ . . . . . . . . . 
. . . . . . . . . . . . . ♛ 
. . . . . . . . ♛ . . . . . 
CPU times: user 672 ms, sys: 1.87 ms, total: 674 ms
Wall time: 723 ms


In [61]:
%%time
at_most_one = at_most_one_seq
at_least_one = at_least_one_seq
exactly_one = exactly_one_seq
solution = nqueens_sat(14)
print("sol : ", solution)
display_nqueens(solution)

sol :  [(0, 13), (1, 4), (2, 9), (3, 7), (4, 3), (5, 10), (6, 12), (7, 2), (8, 0), (9, 11), (10, 6), (11, 8), (12, 5), (13, 1)]
. . . . . . . . . . . . . ♛ 
. . . . ♛ . . . . . . . . . 
. . . . . . . . . ♛ . . . . 
. . . . . . . ♛ . . . . . . 
. . . ♛ . . . . . . . . . . 
. . . . . . . . . . ♛ . . . 
. . . . . . . . . . . . ♛ . 
. . ♛ . . . . . . . . . . . 
♛ . . . . . . . . . . . . . 
. . . . . . . . . . . ♛ . . 
. . . . . . ♛ . . . . . . . 
. . . . . . . . ♛ . . . . . 
. . . . . ♛ . . . . . . . . 
. ♛ . . . . . . . . . . . . 
CPU times: user 402 ms, sys: 2.05 ms, total: 404 ms
Wall time: 404 ms


In [62]:
%%time
at_most_one = at_most_one_bw
at_least_one = at_least_one_bw
exactly_one = exactly_one_bw
solution = nqueens_sat(14)
print("sol : ", solution)
display_nqueens(solution)

sol :  [(0, 6), (1, 0), (2, 12), (3, 1), (4, 8), (5, 10), (6, 2), (7, 4), (8, 9), (9, 11), (10, 3), (11, 5), (12, 7), (13, 13)]
. . . . . . ♛ . . . . . . . 
♛ . . . . . . . . . . . . . 
. . . . . . . . . . . . ♛ . 
. ♛ . . . . . . . . . . . . 
. . . . . . . . ♛ . . . . . 
. . . . . . . . . . ♛ . . . 
. . ♛ . . . . . . . . . . . 
. . . . ♛ . . . . . . . . . 
. . . . . . . . . ♛ . . . . 
. . . . . . . . . . . ♛ . . 
. . . ♛ . . . . . . . . . . 
. . . . . ♛ . . . . . . . . 
. . . . . . . ♛ . . . . . . 
. . . . . . . . . . . . . ♛ 
CPU times: user 601 ms, sys: 1.07 ms, total: 602 ms
Wall time: 605 ms


In [63]:
%%time
at_most_one = at_most_one_he
at_least_one = at_least_one_he
exactly_one = exactly_one_he
display_nqueens(nqueens_sat(14))

. . . . . ♛ . . . ♛ . . ♛ . . . . . . . 
. . . . . . . ♛ . . . ♛ . . . . . . . . 
. . . . . . . . ♛ . . . . ♛ . . . . . . 
. . . . ♛ . ♛ . . . ♛ . . . . . . . . . 
♛ . . . . . . . . . . . . . . . . . . . 
. . . ♛ . . . . . . . . . . . . . . . . 
. ♛ . . . . . . . . . . . . . . . . . . 
. . . ♛ . . . . . . . . . . . . . . . . 
. . ♛ . . . . . . . . . . . . . . . . . 
. . . ♛ . . . . . . . . . . . . . . . . 
. . ♛ . . . . . . . . . . . . . . . . . 
♛ . . . . . . . . . . . . . . . . . . . 
. . ♛ . . . . . . . . . . . . . . . . . 
♛ . . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . . 
CPU times: user 388 ms, sys: 3.18 ms, total: 391 ms
Wall time: 420 ms


## Instances

In [64]:
instances = []

# instance1 = 4
# instances.append(instance1)

# instance2 = 8 
# instances.append(instance2)

instance3 = 16
instances.append(instance3)

instance4 = 32
instances.append(instance4)

# instance5 = 64
# instances.append(instance5)


## Approaches

In [65]:
approaches = ['naive', 'sequential', 'pairwise', 'heule']

def find_optimal(instance):

    # In another preoblem the optimal solution can be found using
    # - linear search
    # - binary search
    # - ...

    # -------------- START LOOKING FOR OPTIMAL SOLUTION --------------
    start_time = time.time()

    # For this simple case we basically find a solution
    sol = nqueens_sat(instance)

    # In Nqueens an optimal solution doesn't exist (always optimal)
    optimal = True

    end_time = time.time()
    elapsed_time = math.floor(end_time - start_time)
    # -------------- END LOOKING FOR OPTIMAL SOLUTION --------------
    
    # In this simple case the objective value function has no sense
    # We can set it to 1 (a dummy value)
    obj = 1

    

    return elapsed_time, optimal, obj, sol

def find_optimal_naive(instance):
    
    at_most_one = at_most_one_np
    at_least_one = at_least_one_np
    exactly_one = exactly_one_np
    
    return find_optimal(instance)

def find_optimal_sequential(instance):

    at_most_one = at_most_one_seq
    at_least_one = at_least_one_seq
    exactly_one = exactly_one_seq

    return find_optimal(instance)

def find_optimal_pairwise(instance):

    at_most_one = at_most_one_bw
    at_least_one = at_least_one_bw
    exactly_one = exactly_one_bw

    return find_optimal(instance)

def find_optimal_heule(instance):

    at_most_one = at_most_one_he
    at_least_one = at_least_one_he
    exactly_one = exactly_one_he

    return find_optimal(instance)

In [66]:
for instance, i in enumerate(instances):

    json_object = {}
    
    for approach in approaches:
        if approach == 'naive':
            elapsed_time, optimal, obj, sol = find_optimal_naive(i)
        elif approach == 'sequential':
            elapsed_time, optimal, obj, sol = find_optimal_sequential(i)
        elif approach == 'pairwise':
            elapsed_time, optimal, obj, sol = find_optimal_pairwise(i)
        elif approach == 'heule':
            elapsed_time, optimal, obj, sol = find_optimal_heule(i)

        json_object [approach] = {
            "time": elapsed_time,
            "optimal": optimal,
            "obj": obj,
            "sol": sol
        }
    
    with open(f"../../res_demo/SAT/{i}.json", "w") as f:
        f.write(json.dumps(json_object, indent=3))